# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [54]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet'])
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jan.krueger\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jan.krueger\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [34]:
# load data from database
engine = create_engine('sqlite:///data/messages.db')
df = pd.read_sql_table('messages', engine)
X = df['message'] 
Y = df.drop(columns=['id', 'message', 'original', 'genre'])
with pd.option_context('display.max_colwidth', 480):
    display(X.sample(20))
display(Y)

6276                                                                                                                                                                                                  last night in the zone latrable, it rain, add us to wet by what we don't even have water has drink however for the tempts. I don't know what to do 
14742                                                                                                                               Just hearing about the magnitude of 7.6 on the Richter scale is enough to terrify someone; imagine those who were actually there when the tremors jolted the earth, says Kim Beam Kon, chairman of the relief centre.
4782                                                                                                                                                                                                     We need help (Centre d'Hbergement Saint Martin) Delmas 4 in the area of rue Saint Martin. We have never rec

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [35]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens
#todo stopwords
#todo stemmer
#todo . , remover

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [45]:
pipeline = Pipeline(
        [
            ('vectorize', CountVectorizer(tokenizer=tokenize)),
            ('tfidtransform', TfidfTransformer()),
            ('classify', MultiOutputClassifier(RandomForestClassifier(), n_jobs=1))
        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [46]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((19662,), (6554,), (19662, 36), (6554, 36))

In [47]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vectorize',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000236EC21BF70>)),
                ('tfidtransform', TfidfTransformer()),
                ('classify',
                 MultiOutputClassifier(estimator=RandomForestClassifier(),
                                       n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [48]:
Y_test_pred = pipeline.predict(X_test)

In [51]:
Y_test_pred = pd.DataFrame(Y_test_pred, columns=Y_test.columns)

In [56]:
Y_test_pred.shape, Y_test.shape

((6554, 36), (6554, 36))

In [71]:
Y_test_pred['related'].value_counts(), Y_test['related'].value_counts()

(1    6014
 0     530
 2      10
 Name: related, dtype: int64,
 1    4930
 0    1568
 2      56
 Name: related, dtype: int64)

In [73]:
for c in Y_test_pred.columns:
    print(c)
    display(classification_report(Y_test[c].to_numpy(), Y_test_pred[c].to_numpy()))
    

related


'              precision    recall  f1-score   support\n\n           0       0.78      0.26      0.40      1568\n           1       0.80      0.98      0.88      4930\n           2       0.50      0.09      0.15        56\n\n    accuracy                           0.80      6554\n   macro avg       0.69      0.44      0.48      6554\nweighted avg       0.79      0.80      0.76      6554\n'

request


'              precision    recall  f1-score   support\n\n           0       0.90      0.99      0.94      5433\n           1       0.91      0.44      0.59      1121\n\n    accuracy                           0.90      6554\n   macro avg       0.90      0.71      0.77      6554\nweighted avg       0.90      0.90      0.88      6554\n'

offer


...

'              precision    recall  f1-score   support\n\n           0       0.98      1.00      0.99      6436\n           1       0.90      0.08      0.14       118\n\n    accuracy                           0.98      6554\n   macro avg       0.94      0.54      0.57      6554\nweighted avg       0.98      0.98      0.98      6554\n'

other_weather


'              precision    recall  f1-score   support\n\n           0       0.95      1.00      0.97      6220\n           1       0.57      0.01      0.02       334\n\n    accuracy                           0.95      6554\n   macro avg       0.76      0.51      0.50      6554\nweighted avg       0.93      0.95      0.93      6554\n'

direct_report


'              precision    recall  f1-score   support\n\n           0       0.86      0.99      0.92      5298\n           1       0.86      0.34      0.49      1256\n\n    accuracy                           0.86      6554\n   macro avg       0.86      0.66      0.70      6554\nweighted avg       0.86      0.86      0.84      6554\n'

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.